In [1]:
# %pip install sklearn

In [2]:
import numpy as np
import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.metrics import r2_score
# from sklearn.model_selection import cross_val_score, KFold
from xgboost import DMatrix


### Input Data - Cleaned Dataset All

In [3]:

# df_TagDesc = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\CSV\Not for Processing\TagDesc.csv')

# df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
# df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# # Concatenate (union) the dataframes
# df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

# df_rC4_1_75 = pd.read_csv(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\IV Optimize\Statistical Testing Sets\rC4_1_75.csv')
# df_rC4_5_75 = pd.read_csv(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\IV Optimize\Statistical Testing Sets\rC4_5_75.csv')

df_rC4_1_75 = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Statistical Testing Sets\rC4_1_75.csv')
df_rC4_5_75 = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Statistical Testing Sets\rC4_5_75.csv')


# Set display options
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
# Assuming the 'Date' column is a string, you can convert it to datetime
df_rC4_1_75['Date'] = pd.to_datetime(df_rC4_1_75['Date'])
# Assuming the 'Date' column is a string, you can convert it to datetime
df_rC4_5_75['Date'] = pd.to_datetime(df_rC4_5_75['Date'])

In [5]:
print(df_rC4_1_75.info())
print(df_rC4_5_75.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47882 entries, 0 to 47881
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                47882 non-null  datetime64[ns]
 1   425_pct_Al          47882 non-null  float64       
 2   C4_pct_Eth          47882 non-null  float64       
 3   C4_pct_H2O          47882 non-null  float64       
 4   HydWtr_pct_Ammonia  47882 non-null  float64       
 5   HydWtr_Na2O         47882 non-null  float64       
 6   Butanol             47882 non-null  float64       
 7   Decanol             47882 non-null  float64       
 8   DI55102             47882 non-null  float64       
 9   DI55152             47882 non-null  float64       
 10  FC55003             47882 non-null  float64       
 11  FC55009             47882 non-null  float64       
 12  FC55552             47882 non-null  float64       
 13  FC55569             47882 non-null  float64   

In [6]:
print(df_rC4_1_75.head())
print(df_rC4_5_75.head())

                 Date  425_pct_Al  C4_pct_Eth  C4_pct_H2O  HydWtr_pct_Ammonia  \
0 2012-05-16 16:00:00        6.31        2.51       21.96                0.91   
1 2012-05-16 18:00:00        6.31        2.51       21.97                0.91   
2 2012-05-16 19:00:00        6.31        2.52       21.98                0.91   
3 2012-05-16 20:00:00        6.31        2.52       21.98                0.90   
4 2012-05-16 21:00:00        6.31        2.52       21.99                0.90   

   HydWtr_Na2O  Butanol  Decanol  DI55102  DI55152  ...  LC55557  LC90366  \
0         2.71    54.86     0.72     0.97     0.93  ...    65.87    61.01   
1         2.81    54.86     0.72     0.97     0.93  ...    65.87    61.01   
2         2.95    54.86     0.72     0.97     0.93  ...    65.87    61.01   
3         3.04    51.19     0.73     0.97     0.92  ...    65.96    56.69   
4         3.13    48.74     0.74     0.97     0.92  ...    65.97    55.18   

   LC90368  PI55004  PI55020  TC55552  TI55021  FF

In [7]:
# Assuming the 'Date' column is already a datetime object
df_rC4_1_75['Year'] = df_rC4_1_75['Date'].dt.year

# Count the records for each year
record_counts_by_year_1_75 = df_rC4_1_75['Year'].value_counts().sort_index()

# Create a DataFrame from the value counts
result_df = pd.DataFrame({
    'Year': record_counts_by_year_1_75.index,
    'Count': record_counts_by_year_1_75.values
})

# Calculate the %Year column
result_df['%Year'] = (result_df['Count'] / (24 * 365))*100

# Output the results
print(result_df)


   Year  Count  %Year
0  2012   2445  27.91
1  2013   6471  73.87
2  2014   5952  67.95
3  2015   5807  66.29
4  2016   6222  71.03
5  2017   6348  72.47
6  2018   6128  69.95
7  2019   4920  56.16
8  2020   3178  36.28
9  2022    411   4.69


# 1 75

In [8]:
# Calculate the differences and create new columns
df_rC4_1_75['FFC55555_diff'] = df_rC4_1_75['FFC55555'] - df_rC4_1_75['FFC55555_adj']
df_rC4_1_75['Butanol_diff'] = df_rC4_1_75['Butanol'] - df_rC4_1_75['Butanol_adj']
df_rC4_1_75['Decanol_diff'] = df_rC4_1_75['Decanol'] - df_rC4_1_75['Decanol_adj']


df_rC4_1_75['rC4_diff'] = (df_rC4_1_75['FFC55555']*df_rC4_1_75['FC55552'])-(df_rC4_1_75['FFC55555_adj']*df_rC4_1_75['FC55552'])

In [9]:

df_rC4_1_75.set_index('Date', inplace=True)


In [10]:
# Sum up the hourly data to get daily totals for each component
daily_totals_1_75 = df_rC4_1_75[['FFC55555_diff','Butanol_diff', 'Decanol_diff', 'rC4_diff']].resample('D').sum()

# Group by year and sum up daily data for each year
daily_totals_by_year_1_75 = df_rC4_1_75[['FFC55555_diff','Butanol_diff', 'Decanol_diff', 'rC4_diff']].groupby(df_rC4_1_75.index.year).resample('D').sum()


In [11]:
# Sum up the hourly data to get daily totals for each component
daily_totals_1_75 = df_rC4_1_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff', 'rC4_diff']].resample('D').sum()

# Group by year and sum up daily data for each year
daily_totals_by_year_1_75 = daily_totals_1_75.groupby(daily_totals_1_75.index.year).sum()

daily_totals_by_year_1_75


,FFC55555_diff,Butanol_diff,Decanol_diff,rC4_diff
Date,,,,
2012,11.31,-972.16,-193.91,378578.34
2013,13.17,-570.83,-68.91,491051.11
2014,5.25,-468.31,-117.13,192716.20
2015,10.65,-72.40,-160.44,377288.31
2016,4.08,-238.11,-144.39,144389.56
2017,2.71,-354.27,-156.57,96017.18
2018,0.17,-450.71,-194.78,6000.78
2019,4.82,-1019.49,-204.87,183830.20
2020,1.88,-299.50,-102.23,66500.48


# 5 75

In [12]:
# Calculate the differences and create new columns
df_rC4_5_75['FFC55555_diff'] = df_rC4_5_75['FFC55555'] - df_rC4_5_75['FFC55555_adj']
df_rC4_5_75['Butanol_diff'] = df_rC4_5_75['Butanol'] - df_rC4_5_75['Butanol_adj']
df_rC4_5_75['Decanol_diff'] = df_rC4_5_75['Decanol'] - df_rC4_5_75['Decanol_adj']

df_rC4_5_75['rC4_diff'] = (df_rC4_5_75['FFC55555']*df_rC4_5_75['FC55552'])-(df_rC4_5_75['FFC55555_adj']*df_rC4_5_75['FC55552'])

In [13]:
df_rC4_5_75.set_index('Date', inplace=True)


In [14]:
# Sum up the hourly data to get daily totals for each component
daily_totals_5_75 = df_rC4_5_75[['FFC55555_diff','Butanol_diff', 'Decanol_diff', 'rC4_diff']].resample('D').sum()

# Group by year and sum up daily data for each year
daily_totals_by_year_5_75 = df_rC4_5_75[['FFC55555_diff','Butanol_diff', 'Decanol_diff', 'rC4_diff']].groupby(df_rC4_5_75.index.year).resample('D').sum()

In [15]:
# Sum up the hourly data to get daily totals for each component
daily_totals_5_75 = df_rC4_5_75[['FFC55555_diff', 'Butanol_diff', 'Decanol_diff', 'rC4_diff']].resample('D').sum()

# Group by year and sum up daily data for each year
daily_totals_by_year_5_75 = daily_totals_5_75.groupby(daily_totals_5_75.index.year).sum()



daily_totals_by_year_5_75

,FFC55555_diff,Butanol_diff,Decanol_diff,rC4_diff
Date,,,,
2012,68.38,-3498.03,-152.43,2380522.01
2013,132.17,-3723.13,43.63,4745483.90
2014,143.32,-8778.80,58.37,5372815.02
2015,130.87,-8454.70,-273.76,4849572.76
2016,129.06,-10143.55,-253.48,4535407.49
2017,154.34,-8551.44,-558.51,5582820.74
2018,16.15,-1841.50,-264.90,548295.22
2019,25.53,-3792.79,-259.97,937712.75
2020,50.12,-6573.67,-177.28,1800929.00
